# Тестовое задание

In [238]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string

In [175]:
count_vect = CountVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
tfidf_vect = TfidfVectorizer(max_df = 0.9, min_df = 10, ngram_range=(1, 3))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
clf = LogisticRegression(solver='liblinear')

## Cкачиваем файл с данными

In [176]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [177]:
#df = pd.read_json ('reviews_Electronics_5.json', lines=True )
#df.to_csv ('Electronics_5.csv', index = None)

# Предобработка

In [178]:
dat = pd.read_csv("Electronics_5.csv")[["asin","reviewText", "overall"]]

## Добавляем бинрную оценку

In [179]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [180]:
dat_t, dat = dat, dat[0:15000]

In [181]:
train_dat, test_dat, train_ans, test_ans = train_test_split(
    dat.reviewText, dat.bin, test_size=0.2)

## Натравим на необработанные данные

In [357]:
count_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
tfidf_vect.fit(dat.reviewText.apply(lambda x: np.str_(x)))
count_train_mtrx_dft = count_vect.transform(train_dat.apply(lambda x: np.str_(x)))
tfidf_train_mtrx_dft = tfidf_vect.transform(train_dat.apply(lambda x: np.str_(x)))
count_test_mtrx_dft = count_vect.transform(test_dat.apply(lambda x: np.str_(x)))
tfidf_test_mtrx_dft = tfidf_vect.transform(test_dat.apply(lambda x: np.str_(x)))

## Приводим к базовой форме

In [183]:
train_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), train_dat.apply(lambda x: np.str_(x)))))
test_dat_tr = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), test_dat.apply(lambda x: np.str_(x)))))
dat["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), dat.reviewText.apply(lambda x: np.str_(x)))))

/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Можно воспользоваться анализатором, который предоставляют векторайзеры

In [184]:
# anlz = count_vect.build_analyzer()
# dat.words = list(map(lambda s: (" ").join(anlz(s)), dat.reviewText))

## Натравим на обработанные данные

In [356]:
count_vect.fit(dat.words)
tfidf_vect.fit(dat.words)
count_train_mtrx = count_vect.transform(train_dat_tr)
tfidf_train_mtrx = tfidf_vect.transform(train_dat_tr)
count_test_mtrx = count_vect.transform(test_dat_tr)
tfidf_test_mtrx = tfidf_vect.transform(test_dat_tr)

## Проверим, что получилось

In [186]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,True,we got thi gp for my husband who is an otr ove...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,False,profession otr truck driver and bought tnd 700...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,False,well what can say ve had thi unit in my truck ...
3,0528881469,"Not going to write a long review, even thought...",2,False,not go to write long review even thought thi u...
4,0528881469,I've had mine for a year and here's what we go...,1,False,ve had mine for year and here what we got it t...
...,...,...,...,...,...
14995,B00004ZCJE,I bought this filter primarily for protecting ...,5,True,bought thi filter primarili for protect the fr...
14996,B00004ZCJE,Got if for my 50mm lens to protect it.The fit ...,3,False,got if for my 50mm len to protect it the fit i...
14997,B00004ZCJE,I wanted a lens to protect my expensive Nikon ...,5,True,want len to protect my expens nikon larger len...
14998,B00004ZCJE,I am comparing this with an multi-coated UV fi...,3,False,am compar thi with an multi coat uv filter tha...


## Разобьем вектора на выборки

In [187]:
# count_train_mtrx, count_test_mtrx, count_train_ans, count_test_ans = train_test_split(
#     dat_count, dat.bin, test_size=0.2)
# #dat_count.toarray()

In [188]:
# tfidf_train_mtrx, tfidf_test_mtrx, tfidf_train_ans, tfidf_test_ans = train_test_split(
#     dat_tfidf, dat.bin, test_size=0.2)
# #dat_tfidf.toarray()

In [189]:
# count_train_mtrx_dft, count_test_mtrx_dft, count_train_ans_dft, count_test_ans_dft = train_test_split(
#     dat_count_deft, dat.bin, test_size=0.2)

In [190]:
# tfidf_train_mtrx_dft, tfidf_test_mtrx_dft, tfidf_train_ans_dft, tfidf_test_ans_dft = train_test_split(
#     dat_tfidf_deft, dat.bin, test_size=0.2)

# Обучим модели и посмотрим их точность

## Обработанные

In [191]:
clf.fit(count_train_mtrx, train_ans)
cout_pred = clf.predict(count_test_mtrx)
accuracy_score(cout_pred, test_ans)

0.8746666666666667

In [192]:
clf.fit(tfidf_train_mtrx, train_ans)
tfidf_pred = clf.predict(tfidf_test_mtrx)
accuracy_score(tfidf_pred, test_ans)

0.8543333333333333

## Дефолтные

In [193]:
clf.fit(count_train_mtrx_dft, train_ans)
cout_pred = clf.predict(count_test_mtrx_dft)
accuracy_score(cout_pred, test_ans)

0.8743333333333333

In [194]:
clf.fit(tfidf_train_mtrx_dft, train_ans)
tfidf_pred = clf.predict(tfidf_test_mtrx_dft)
accuracy_score(tfidf_pred, test_ans)

0.8513333333333334

## Обработанные с оптимальными параметрами

In [195]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx, train_ans)
print(search.best_params_)

{'C': 0.1, 'penalty': 'l2'}


In [196]:
accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx))

0.8746666666666667

In [344]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx,  train_ans)
print(search.best_params_)

{'C': 2.8000000000000003, 'penalty': 'l1'}


In [345]:
accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx))

0.88

## Дефолтные с оптимальными параметрами

In [199]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(count_train_mtrx_dft, train_ans)
print(search.best_params_)

{'C': 0.2, 'penalty': 'l2'}


In [200]:
accuracy_score(test_ans, search.best_estimator_.predict(count_test_mtrx_dft))

0.8733333333333333

In [352]:
param_grid = {
    'C': np.arange(0.1, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(tfidf_train_mtrx_dft, train_ans)
print(search.best_params_)

{'C': 2.6, 'penalty': 'l1'}


In [353]:
accuracy_score(test_ans, search.best_estimator_.predict(tfidf_test_mtrx_dft))

0.8753333333333333

## Проверим баланс классов

In [203]:
print("All :\n1 - ", sum(dat.bin), "\n0 - ", len(dat.bin) - sum(dat.bin)) 
print("Train :\n1 - ", sum(train_ans), "\n0 - ", len(train_ans) - sum(train_ans)) 
print("Test :\n1 - ", sum(test_ans), "\n0 - ", len(test_ans) - sum(test_ans)) 

All :
1 -  12390 
0 -  2610
Train :
1 -  9908 
0 -  2092
Test :
1 -  2482 
0 -  518


## Возьмем новый набор данных

In [280]:
dat2 = pd.read_json("Tools_and_Home_Improvement_5.json", lines = True)[["reviewText", "overall"]]

In [281]:
dat2["bin"] = dat2.overall > 3

## Случайный выбор

In [339]:
rat = (len(dat2.bin)-sum(dat2.bin)) / sum(dat2.bin)
print("Соотношение false/true:",len(dat2.bin)-sum(dat2.bin), " / ", sum(dat2.bin))

Соотношение false/true: 20874  /  113602


In [340]:
dat2["Choice"] = 0
for i in range(len(dat2.bin)):
    dat2.Choice[i] = (np.random.randint(round(10 / rat)) if dat2.bin[i] else np.random.randint(round(10)))

/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [341]:
new_test_dat = dat2.loc[dat2.Choice == 0][["reviewText", "overall", "bin"]]
new_test_dat

,reviewText,overall,bin
26,This light is useless for reading in bed next ...,2,False
56,These burn out really fast. I buy them becaus...,1,False
75,Still looking for the 'dimmable' option--only ...,4,True
83,My desk lamp was a bust. Imagine my surprise w...,1,False
201,While the motor and blade assembly of this too...,3,False
...,...,...,...
134405,Need to get a stuck bolt off? These pliers hav...,4,True
134422,This is my second rechargeable lantern by Ivat...,5,True
134446,It's a very high standard cigar scissor that i...,5,True
134451,"My husband loves this punch, super easy to use...",5,True


In [342]:
print("Соотношение false/true:",len(new_test_dat.bin)-sum(new_test_dat.bin), " / ", sum(new_test_dat.bin))

Соотношение false/true: 2071  /  2066


## Без обработки

In [359]:
accuracy_score(new_test_dat.bin, search.best_estimator_.predict(tfidf_vect.transform(new_test_dat.reviewText)))

0.6925308194343728